In [1]:
from easyquant import MongoIo
from easyquant.indicator.base import *
import easyquotation
import math
import numpy as np
import pandas as pd
from easyquant import MongoIo
# from easyquant.qafetch import QATdx as tdx
from easyquant.indicator.base import *
import pandas as pd
from easyquant import EasyTime
from datetime import datetime, time
from ctypes import * 
# import datetime
import math
import numpy as np
import pandas as pd
import talib
from ctypes import * 
import QUANTAXIS as QA

# import time

In [3]:
m=MongoIo()
source=easyquotation.use('sina')
table = 'st_orders-%s' % 'tdx_yhzc'

In [4]:

def new_df(df_day, data, now_price):
    code = data['code']
    # now_vol = data['volume']
    last_time = pd.to_datetime(data['datetime'][0:10])
    # print("code=%s, data=%s" % (self.code, self._data['datetime']))
    # df_day = data_buf_day[code]
    # df_day.loc[last_time]=[0 for x in range(len(df_day.columns))]
    df_day.at[(last_time, code), 'open'] = data['open']
    df_day.at[(last_time, code), 'high'] = data['high']
    df_day.at[(last_time, code), 'low'] = data['low']
    df_day.at[(last_time, code), 'close'] = now_price
    df_day.at[(last_time, code), 'volume'] = data['volume']
    df_day.at[(last_time, code), 'amount'] = data['amount']
    return df_day


In [5]:
def tdx_dqe_cfc_A1(data, sort=False):
    # 选择／排序
    C = data.close
    O = data.open
    JC =IF(ISLASTBAR(C), O, C)
    MC = (0.3609454219 * JC - 0.03309329629 * REF(C, 1) - 0.04241822779 * REF(C, 2) - 0.026737249 * REF(C, 3) \
           - 0.007010041271 * REF(C, 4) - 0.002652859952 * REF(C, 5) - 0.0008415042966 * REF(C, 6) \
           - 0.0002891931964 * REF(C, 7) - 0.0000956265934 * REF(C, 8) - 0.0000321286052 * REF(C, 9) \
           - 0.0000106773454 * REF(C, 10) - 0.0000035457562 * REF(C, 11) -- 0.0000011670713 * REF(C, 12)) / (1 - 0.7522406533)
    # 竞价涨幅 := (DYNAINFO(4) / DYNAINFO(3) - 1) * 100;
    竞价涨幅 = (C / REF(C, 1) - 1) * 100
    # ST := STRFIND(stkname, 'ST', 1) > 0;
    # S := STRFIND(stkname, 'S', 1) > 0;
    # 停牌 := (DYNAINFO(4)=0);
    #
    # 附加条件 := (not (ST) and not (S) and NOT(停牌)) * (竞价涨幅 < 9.85) * (竞价涨幅 > (0));
    附加条件 = IFAND(竞价涨幅 < 9.85, 竞价涨幅 > 0, 1, 0)
#     if sort:
    刀 = (MC - JC) / JC * 1000
#     else:
#         刀 = (MC - JC) / JC * 1000 * 附加条件

    return 刀

In [6]:
def tdx_dqe_cfc_A12(data, sort=False):
    # 选择／排序
    C = data.close
    O = data.open
    JC = IF(ISLASTBAR(C), O, C)
    MC = (0.3609454219 * JC - 0.03309329629 * REF(C, 1) - 0.04241822779 * REF(C, 2) - 0.026737249 * REF(C, 3) \
           - 0.007010041271 * REF(C, 4) - 0.002652859952 * REF(C, 5) - 0.0008415042966 * REF(C, 6) \
           - 0.0002891931964 * REF(C, 7) - 0.0000956265934 * REF(C, 8) - 0.0000321286052 * REF(C, 9) \
           - 0.0000106773454 * REF(C, 10) - 0.0000035457562 * REF(C, 11) -- 0.0000011670713 * REF(C, 12)) / (1 - 0.7522406533)
    # 竞价涨幅 := (DYNAINFO(4) / DYNAINFO(3) - 1) * 100;
    竞价涨幅 = (O / REF(C, 1) - 1) * 100
    # ST := STRFIND(stkname, 'ST', 1) > 0;
    # S := STRFIND(stkname, 'S', 1) > 0;
    # 停牌 := (DYNAINFO(4)=0);
    #
    # 附加条件 := (not (ST) and not (S) and NOT(停牌)) * (竞价涨幅 < 9.85) * (竞价涨幅 > (0));
    附加条件 = IFAND(竞价涨幅 < 9.85, 竞价涨幅 > 0, 1, 0)
#     if sort:
    刀 = (MC - JC) / JC * 1000
#     else:
#         刀 = (MC - JC) / JC * 1000 * 附加条件

    return 刀

In [63]:

def tdx_yhzc(data):
    # 用户注册
    # pass
    # C = data.close
    CLOSE = data.close
    # HIGH = data.high
    # H = data.high
    # L = data.low
    # LOW = data.low
    # OPEN = data.open
    # O = data.open
    VOL = data.volume
    # AMOUNT = data.amount

    # 除业绩后退股 := FINANCE(30) >= REF(FINANCE(30), 130);
    # D0 := 除业绩后退股;
    # D2 := IF(NAMELIKE('S'), 0, 1);
    # D3 := IF(NAMELIKE('*'), 0, 1);
    # D4 := DYNAINFO(17) > 0;
    # 去除大盘股 := CAPITAL / 1000000 < 20;
    # 去高价 := C <= 60;
    # 去掉 := D0 and D2 and D3 and D4 and 去除大盘股 and 去高价 and NOT(C >= REF(C, 1) * 1.097 and C = O and H = L);
    TJ_V = VOL > 3 * MA(VOL,89)
    DIF1 = (EMA(CLOSE, 12) - EMA(CLOSE, 26)) / EMA(CLOSE, 26) * 100
    DEA1 = EMA(DIF1, 9)
    AAA1 = (DIF1 - DEA1) * 100
    # MA120 = REF(MA(C,120),1)
    # MA5 = REF(MA(C, 120),1)
    # MA10 = REF(MA(C, 120),1)
    # PTGD = REF(HHV(C,120),1)
    # XIN_GAO = IFAND(C > PTGD, C > MA120, True, False)
    用 = 45
    户 = AAA1 - REF(AAA1, 1)
    注册 = CROSS(户, 用)
    DIF = (EMA(CLOSE, 10) - EMA(CLOSE, 72)) / EMA(CLOSE, 72) * 100
    DEA = EMA(DIF, 17)
    AAA = (DIF - DEA) * 100
    用户 = CROSS(AAA - REF(AAA, 1), 45)
    # 用户注册 = IFAND4(注册 , 用户, TJ_V, XIN_GAO, 1, 0) #and 去掉;
    用户注册 = IFAND3(注册 , 用户, TJ_V, 1, 0) #and 去掉;
    return 用户注册


In [119]:
# today
code='300787'
data=m.get_stock_day(code)
newdata=source.stocks(code)[code]
now_price = newdata['open']
data = new_df(data.copy(), newdata, now_price)
# data.iat[-1,3]=newdata['open']
data.tail()
C = data.close
O = data.open
V = data.volume
H = data.high
L = data.low


In [120]:
TJ1=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False)
TJ2=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0)
ccc=tdx_dqe_cfc_A1(data)
yhzc=tdx_yhzc(data)
data['A1']=ccc
data['yhzc']=yhzc
data['tj1']=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False)
data['tj2']=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0)
# result
data.tail()


,,open,high,low,close,volume,amount,A1,yhzc,tj1,tj2
date,code,,,,,,,,,,
2020-11-03,300787,46.34,47.58,45.66,46.73,2.683100e+04,125587848.0,-4.134629,0,False,True
2020-11-04,300787,46.88,47.67,46.29,47.31,3.035000e+04,142469968.0,6.291537,0,False,True
2020-11-05,300787,48.36,49.15,47.50,48.48,3.927800e+04,190026400.0,15.684385,0,True,True
2020-11-06,300787,48.39,48.65,46.59,46.87,3.400800e+04,161310304.0,-5.548653,0,True,True
2020-11-09,300787,47.80,50.88,47.80,47.80,2.156544e+08,4322145.0,2.342464,0,True,True


In [58]:
# back:
code='002242'
data=m.get_stock_day(code, st_end='2020-11-05')

C = data.close
O = data.open
V = data.volume
H = data.high
L = data.low
# data.tail()

In [59]:
data=data.copy()
ccc=tdx_dqe_cfc_A12(data).tail()
data['A1']=ccc
data['tj1']=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False)
data['tj2']=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0)
# result
data.tail()

,,open,high,low,close,volume,amount,A1,tj1,tj2
date,code,,,,,,,,,
2020-10-30,002242,36.65,36.70,34.81,35.36,81318.0,290368544.0,-40.651927,False,True
2020-11-02,002242,35.10,36.29,35.10,35.83,58381.0,208967888.0,-14.172844,False,True
2020-11-03,002242,35.83,36.74,35.19,36.32,67585.0,243814624.0,2.036720,False,True
2020-11-04,002242,36.18,36.94,36.00,36.82,57094.0,208352560.0,9.963941,False,True
2020-11-05,002242,37.03,37.70,36.55,37.00,51057.0,189177312.0,8.671438,False,True


In [223]:
code='300612'
dataInfo = m.get_stock_info(code)
dataInfo.jinglirun[0]

-112066005.859375

In [211]:
300364	7.62	8.49	7.38	7.92	787380.0	621797440.0	48.325356
300464	20.50	22.88	20.08	22.30	249674.0	533748384.0	0.000000	True	True
300689	48.51	53.56	47.11	49.70	51398.0	258483824.0	25.845675

SyntaxError: invalid syntax (<ipython-input-211-d05762d70d5f>, line 1)

In [ ]:
dataInfo = mongo.get_stock_info(code)